<a href="https://colab.research.google.com/github/Ki17ua/Sustainable-Group-7/blob/main/new_latest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import folium
from scipy.spatial import Voronoi

# Load data from the FDNY Firehouse listing CSV
try:
    df = pd.read_csv('D:/Chrome/FDNY_Firehouse_Listing_20240508.csv')
    df['Latitude'] = pd.to_numeric(df['Latitude'], errors='coerce')
    df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce')
    fdny_coordinates = df[['Latitude', 'Longitude']].dropna()
except Exception as e:
    print(f"Error reading or parsing CSV file: {e}")

# Constants for conversion from miles to degrees
miles_per_degree_lat = 69
miles_per_degree_lon = 53

# Original substations data
substations = [
    (40.802503, -73.963368, "Substation 1", 3),
    (40.765160, -73.984227, "Substation 2", 2),
    (40.761372, -73.968100, "Substation 3", 2),
    (40.7869998, -73.9482143, "Substation 4", 0),
    (40.8155481, -73.9480725, "Substation 5", 3),
    (40.8244064, -73.8934206, "Substation 6", 5),
    (40.7705348, -73.8670148, "Substation 7", 5),
    (40.8074422, -73.9365556, "Substation 8", 0),
    (40.8105296, -73.9232252, "Substation 9", 0),
    (40.8079077, -73.9061065, "Substation 10", 0),
    (40.765686, -73.900183, "Substation 11", 5)
]

# Create the map centered around New York City
map_nyc = folium.Map(location=[40.785091, -73.968285], zoom_start=12)

# Prepare the points for Voronoi diagram using only substations
points = np.array([[sub[1], sub[0]] for sub in substations])
vor = Voronoi(points)

# Add Voronoi diagram to the map
for region in vor.regions:
    if -1 not in region and region:
        polygon = [vor.vertices[i] for i in region]
        folium.Polygon(
            locations=[(p[1], p[0]) for p in polygon],
            color='grey',
            fill=True,
            fill_opacity=0.5
        ).add_to(map_nyc)

# Function to compute square offsets based on square miles area
def compute_offsets(area_miles):
    side_length_miles = np.sqrt(area_miles)
    lat_offset = (side_length_miles / 2) / miles_per_degree_lat
    lon_offset = (side_length_miles / 2) / miles_per_degree_lon
    return lat_offset, lon_offset

# Add squares based on specified areas for substations
for lat, lon, label, area in substations:
    if area > 0:
        lat_offset, lon_offset = compute_offsets(area)
        square = [
            (lat + lat_offset, lon - lon_offset),
            (lat + lat_offset, lon + lon_offset),
            (lat - lat_offset, lon + lon_offset),
            (lat - lat_offset, lon - lon_offset)
        ]
        folium.Polygon(
            locations=square,
            color='grey',
            fill=True,
            fill_color='grey',
            fill_opacity=0.5,
            tooltip=label
        ).add_to(map_nyc)

# Add markers for original substations with red icons
for lat, lon, label, _ in substations:
    folium.Marker(
        [lat, lon],
        popup=f"<strong>{label}</strong>",
        tooltip=label,
        icon=folium.Icon(color='red', prefix='fa', icon='bolt')
    ).add_to(map_nyc)

# Add markers for new FDNY coordinates with blue icons
for index, row in fdny_coordinates.iterrows():
    folium.Marker(
        [row['Latitude'], row['Longitude']],
        popup=f"FDNY Firehouse {index+1}",
        icon=folium.Icon(color='blue', icon_size=(5, 5))
    ).add_to(map_nyc)

# Save the map to a file
map_nyc.save('D:/Columbia/sustainable/NYC_Substations_and_Firehouses.html')


In [ ]:
import numpy as np
import pandas as pd
import folium
from scipy.spatial import Voronoi
from shapely.wkt import loads

# Load data from the police stations CSV
try:
    police_data = pd.read_csv('D:/Chrome/nypp.csv')
    # Extract the centroid of each precinct's geometry
    police_data['centroid'] = police_data['the_geom'].apply(lambda x: loads(x).centroid)
    # Create columns for latitude and longitude from the centroid points
    police_data['Latitude'] = police_data['centroid'].apply(lambda x: x.y)
    police_data['Longitude'] = police_data['centroid'].apply(lambda x: x.x)
except Exception as e:
    print(f"Error reading or parsing CSV file: {e}")

# Constants for conversion from miles to degrees
miles_per_degree_lat = 69
miles_per_degree_lon = 53

# Original substations data
substations = [
    (40.802503, -73.963368, "Substation 1", 3),
    (40.765160, -73.984227, "Substation 2", 2),
    (40.761372, -73.968100, "Substation 3", 2),
    (40.7869998, -73.9482143, "Substation 4", 0),
    (40.8155481, -73.9480725, "Substation 5", 3),
    (40.8244064, -73.8934206, "Substation 6", 5),
    (40.7705348, -73.8670148, "Substation 7", 5),
    (40.8074422, -73.9365556, "Substation 8", 0),
    (40.8105296, -73.9232252, "Substation 9", 0),
    (40.8079077, -73.9061065, "Substation 10", 0),
    (40.765686, -73.900183, "Substation 11", 5)
]

# Function to compute square offsets based on square miles area
def compute_offsets(area_miles):
    side_length_miles = np.sqrt(area_miles)
    lat_offset = (side_length_miles / 2) / miles_per_degree_lat
    lon_offset = (side_length_miles / 2) / miles_per_degree_lon
    return lat_offset, lon_offset

# Create the map centered around New York City
map_nyc = folium.Map(location=[40.785091, -73.968285], zoom_start=12)

# Prepare the points for Voronoi diagram using only substations
points = np.array([[sub[1], sub[0]] for sub in substations])
vor = Voronoi(points)

# Add Voronoi diagram to the map
for region in vor.regions:
    if -1 not in region and region:
        polygon = [vor.vertices[i] for i in region]
        folium.Polygon(
            locations=[(p[1], p[0]) for p in polygon],
            color='grey',
            fill=True,
            fill_opacity=0.5
        ).add_to(map_nyc)

# Add squares based on specified areas for substations
for lat, lon, label, area in substations:
    if area > 0:
        lat_offset, lon_offset = compute_offsets(area)
        square = [
            (lat + lat_offset, lon - lon_offset),
            (lat + lat_offset, lon + lon_offset),
            (lat - lat_offset, lon + lon_offset),
            (lat - lat_offset, lon - lon_offset)
        ]
        folium.Polygon(
            locations=square,
            color='grey',
            fill=True,
            fill_color='grey',
            fill_opacity=0.7,
            tooltip=label
        ).add_to(map_nyc)

# Add markers for original substations with red icons
for lat, lon, label, _ in substations:
    folium.Marker(
        [lat, lon],
        popup=f"<strong>{label}</strong>",
        tooltip=label,
        icon=folium.Icon(color='red', prefix='fa', icon='bolt')
    ).add_to(map_nyc)

# Add markers for new police station coordinates with blue icons
for index, row in police_data.iterrows():
    folium.Marker(
        [row['Latitude'], row['Longitude']],
        popup=f"Police Precinct {row['Precinct']}",
        icon=folium.Icon(color='blue', icon_size=(5, 5))
    ).add_to(map_nyc)

# Save the map
map_nyc.save('D:/Columbia/sustainable/NYC_Substations_and_Police_Stations_Map.html')


In [ ]:
import numpy as np
import pandas as pd
import folium
from scipy.spatial import Voronoi

# Load data from CSV
try:
    data = pd.read_csv('D:/Chrome/Health_Facility_Map.csv')
    # Filter for entries with "Hospital" in the description
    hospital_data = data[data['Description'].str.contains("Hospital", na=False)]
    # Convert to numeric and handle NaN values properly
    hospital_data.loc[:, 'Facility Latitude'] = pd.to_numeric(hospital_data['Facility Latitude'], errors='coerce')
    hospital_data.loc[:, 'Facility Longitude'] = pd.to_numeric(hospital_data['Facility Longitude'], errors='coerce')
    # Drop rows with NaN coordinates
    hospital_data = hospital_data.dropna(subset=['Facility Latitude', 'Facility Longitude'])
except Exception as e:
    print(f"Error reading or parsing CSV file: {e}")

if not hospital_data.empty:
    # Constants for conversion from miles to degrees
    miles_per_degree_lat = 69
    miles_per_degree_lon = 53

    # Substations data
    substations = [
        (40.802503, -73.963368, "Substation 1", 3),
        (40.765160, -73.984227, "Substation 2", 2),
        (40.761372, -73.968100, "Substation 3", 2),
        (40.7869998, -73.9482143, "Substation 4", 0),
        (40.8155481, -73.9480725, "Substation 5", 3),
        (40.8244064, -73.8934206, "Substation 6", 5),
        (40.7705348, -73.8670148, "Substation 7", 5),
        (40.8074422, -73.9365556, "Substation 8", 0),
        (40.8105296, -73.9232252, "Substation 9", 0),
        (40.8079077, -73.9061065, "Substation 10", 0),
        (40.765686, -73.900183, "Substation 11", 5)
    ]

    # Create the map
    map_nyc = folium.Map(location=[40.785091, -73.968285], zoom_start=12)

    # Voronoi diagram for substations
    points = np.array([[sub[1], sub[0]] for sub in substations])
    vor = Voronoi(points)
    for region in vor.regions:
        if -1 not in region and region:
            polygon = [vor.vertices[i] for i in region]
            folium.Polygon(
                locations=[(p[1], p[0]) for p in polygon],
                color='grey',
                fill=True,
                fill_opacity=0.5
            ).add_to(map_nyc)

    # Function to compute square offsets
    def compute_offsets(area_miles):
        side_length_miles = np.sqrt(area_miles)
        lat_offset = (side_length_miles / 2) / miles_per_degree_lat
        lon_offset = (side_length_miles / 2) / miles_per_degree_lon
        return lat_offset, lon_offset

    # Add squares for substations
    for lat, lon, label, area in substations:
        if area > 0:
            lat_offset, lon_offset = compute_offsets(area)
            square = [
                (lat + lat_offset, lon - lon_offset),
                (lat + lat_offset, lon + lon_offset),
                (lat - lat_offset, lon + lon_offset),
                (lat - lat_offset, lon - lon_offset)
            ]
            folium.Polygon(
                locations=square,
                color='grey',
                fill=True,
                fill_color='grey',
                fill_opacity=0.5,
                tooltip=label
            ).add_to(map_nyc)

# Add markers for original substations with red icons
    for lat, lon, label, _ in substations:
      folium.Marker(
          [lat, lon],
          popup=f"<strong>{label}</strong>",
          tooltip=label,
          icon=folium.Icon(color='red', prefix='fa', icon='bolt')
    ).add_to(map_nyc)
    # Add markers for hospitals
    for index, row in hospital_data.iterrows():
        folium.Marker(
            [row['Facility Latitude'], row['Facility Longitude']],
            popup=f"Hospital: {row['Facility Name']}",
            icon=folium.Icon(color='blue', icon_size=(5, 5))
        ).add_to(map_nyc)

    # Save the map
    map_nyc.save('D:/Columbia/sustainable/NYC_Substations_and_Hospitals_Map.html')

else:
    print("No data to display on the map.")
